# **TradeCare: Data Collection Notebook**

## Objectives
* Fetch historical Bitcoin OHLCV (Open, High, Low, Close, Volume) data from a GitHub-hosted repository that provides automated daily updates.
* Verify data loaded correctly (basic checks)
* Save raw data for next phase (cleaning)
* Document data source and collection method

## Inputs

*  **Data Source:** GitHub Repository (automated updates)
*   **URL:** https://raw.githubusercontent.com/mouadja02/bitcoin-hourly-ohclv-dataset/main/btc-hourly-price_2015_2025.csv\n",
*   **Asset:** BTC-USD
*   **Timeframe:** 1 Hour
*   **Period:** 2015 - present

## Outputs

* `inputs/datasets/raw/bitcoin_hourly_raw.csv` - RAW data (uncleaned)
* `outputs/data_collection_report.txt` - Collection summary

## Additional Comments

This GitHub dataset provides a **unique combination** rarely found in ML projects:

* **Fresh & Maintained:** Automated workflow fetches current data from CryptoCompare API daily and stores backups on GitHub. Repository contains Bitcoin hourly price data from 2015 to present with continuous updates
* **Simple**: Direct CSV access via single URL
* **Free**: No API keys or costs  
* **Reliable**: No rate limits or auth failures  
* **Transparent**: Git history shows every change  
* **Scalable**: Should work in production environments  


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1

Section 1 content

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
